### 20184645 김영찬
### 기상청 데이터 응용 예

In [53]:
import urllib.request
from bs4 import BeautifulSoup

page = urllib.request.urlopen("http://www.weather.go.kr/weather/observation/currentweather.jsp").read()
soup = BeautifulSoup(page, 'html.parser')

In [54]:
import sqlite3
conn = sqlite3.connect('Weather.db')
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

In [55]:
current_time = soup.find('input',{'class':'time'})
dt = current_time['value'].split('.')
year = dt[0]
month = dt[1]
date = dt[2]
time = dt[3]
date_time = year+"-"+month+"-"+date+" "+time
print(date_time)

2021-10-05 22:00


In [56]:
table = soup.find('table',{'class':'table_develop3'})
data = []

In [57]:
for tr in table.find_all('tr'):
    tds = list(tr.find_all('td'))
    for td in tds:
        if td.find('a'):
            city = td.find('a').text
            temperature = tds[5].text
            humidity = tds[9].text
            data.append([city,temperature,humidity])
            
            cursor.execute('''
            insert into CityWeather (city, temperature, humidity, dtime) values(?,?,?,?)
            ''',  (city, temperature, humidity, date_time))
conn.commit()

In [58]:
cursor.execute('select * from CityWeather where temperature > 20')
rows=cursor.fetchall()

for row in rows:
    print("도시 : %s, 온도: %s, 습도: %s, 시간: %s"%
          (row['city'],row['temperature'],row['humidity'],row['dtime']))
    
conn.close()

도시 : 강진군, 온도: 22.4, 습도: 93, 시간: 2021-10-05 21:00
도시 : 강화, 온도: 20.7, 습도: 97, 시간: 2021-10-05 21:00
도시 : 거제, 온도: 22.4, 습도: 87, 시간: 2021-10-05 21:00
도시 : 경주시, 온도: 20.7, 습도: 92, 시간: 2021-10-05 21:00
도시 : 고산, 온도: 24.7, 습도: 90, 시간: 2021-10-05 21:00
도시 : 고창, 온도: 20.5, 습도: 89, 시간: 2021-10-05 21:00
도시 : 고창군, 온도: 20.1, 습도: 81, 시간: 2021-10-05 21:00
도시 : 고흥, 온도: 20.7, 습도: 97, 시간: 2021-10-05 21:00
도시 : 광양시, 온도: 23.1, 습도: 88, 시간: 2021-10-05 21:00
도시 : 광주, 온도: 23.3, 습도: 85, 시간: 2021-10-05 21:00
도시 : 구미, 온도: 20.3, 습도: 82, 시간: 2021-10-05 21:00
도시 : 군산, 온도: 20.6, 습도: 99, 시간: 2021-10-05 21:00
도시 : 김해시, 온도: 23.3, 습도: 86, 시간: 2021-10-05 21:00
도시 : 남원, 온도: 20.5, 습도: 90, 시간: 2021-10-05 21:00
도시 : 남해, 온도: 22.3, 습도: 90, 시간: 2021-10-05 21:00
도시 : 대구, 온도: 22.4, 습도: 78, 시간: 2021-10-05 21:00
도시 : 대전, 온도: 21.8, 습도: 87, 시간: 2021-10-05 21:00
도시 : 목포, 온도: 23.1, 습도: 86, 시간: 2021-10-05 21:00
도시 : 밀양, 온도: 21.5, 습도: 86, 시간: 2021-10-05 21:00
도시 : 보령, 온도: 21.0, 습도: 91, 시간: 2021-10-05 21:00
도시 : 보성군, 온도: 20.8, 습도: 93, 시간: 202